In [53]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime, timedelta
import re

In [54]:
# açao br e fii
df_acfi = pd.read_excel("../input/pat.xlsx", sheet_name="AçoesFII")
df_acfi = df_acfi.rename({"DATA": "date","ITEM":"stock"}, axis=1)
df_acfi["date"] = pd.to_datetime(df_acfi["date"])
df_acfi = df_acfi.drop(["TAXA TOTAL"],axis=1)	
df_acfi.columns = df_acfi.columns.str.lower()
# açao de fora
df_ac_int = pd.read_excel("../input/pat.xlsx", sheet_name="Stocks")
df_ac_int = df_ac_int.rename({"DATA": "date","ITEM":"stock"}, axis=1)
df_ac_int["date"] = pd.to_datetime(df_ac_int["date"])
df_ac_int.columns = df_ac_int.columns.str.lower()
# moeda fora
df_usd = pd.read_csv(
    "../data/downloaded/data_bench_cur/currencies_historical_data.csv", sep=","
)
# arrumando
df_usd = df_usd.drop(["EUR_bid", "EUR_ask","index"], axis=1).rename(
    {"USD_bid": "USD_compra", "USD_ask": "USD_venda","ITEM":"stock"}, axis=1
)[2:]
df_usd["Date"] = pd.to_datetime(df_usd["Date"])
df_usd.columns = df_usd.columns.str.lower()
df_usd["month"] = df_usd["date"].dt.month
df_usd["year"] = df_usd["date"].dt.year
df_usd

,date,usd_compra,usd_venda,month,year
2,2000-01-03,1.8003,1.8011,1,2000
3,2000-01-04,1.8329,1.8337,1,2000
4,2000-01-05,1.8536,1.8544,1,2000
5,2000-01-06,1.8453,1.8461,1,2000
6,2000-01-07,1.8273,1.8281,1,2000
...,...,...,...,...,...
8787,2024-01-22,4.9484,4.9490,1,2024
8788,2024-01-23,4.9709,4.9715,1,2024
8789,2024-01-24,4.9186,4.9192,1,2024
8790,2024-01-25,4.9226,4.9232,1,2024


In [55]:
df_acfi.head()

,date,tipo,stock,qtd,preço médio,total
0,2022-03-10,ETF,SMAL11.SA,11,110.00,1210.000000
1,2022-03-10,ETF,BOVA11.SA,11,109.92,1209.120000
2,2022-03-10,ETF,SMAL11.SA,23,111.37,2561.510000
3,2022-03-10,ETF,BOVA11.SA,23,112.82,2594.860000
4,2024-01-05,ETF,DIVO11.SA,11,90.58,996.716667


In [56]:
df_ac_int["preço médio"] = df_ac_int["preço médio_usd"] * df_ac_int["cambio_venda"]
df_ac_int["total"] = df_ac_int["total_usd"] * df_ac_int["cambio_venda"]
#df_ac_int["taxa total"] = np.nan
df_ac_int = df_ac_int.drop(["preço médio_usd", "total_usd", "cambio_venda"], axis=1)
df_ac_int.head()

,date,tipo,stock,qtd,preço médio,total
0,2022-10-18,ETF,EIMI.L,2,148.394752,296.789504
1,2022-10-18,ETF,VWRA.L,1,502.676683,502.676683
2,2022-11-30,ETF,VWRA.L,5,529.357059,2646.785295
3,2023-01-16,ETF,VWRA.L,11,523.129497,5754.424472
4,2023-02-16,ETF,VWRA.L,5,553.891648,2769.458240


In [57]:
df_variavel = pd.concat([df_acfi, df_ac_int]).sort_values("date")
# Convert 'date' columns to datetime objects
df_variavel['date'] = pd.to_datetime(df_variavel['date'])
#df_variavel["month"] = df_variavel["date"].dt.month
#df_variavel["year"] = df_variavel["date"].dt.year

#df_variavel['cumsum_total'] = df_variavel.groupby(['stock'])['total'].cumsum()
#df_variavel['cumsum_qtd'] = df_variavel.groupby(['stock'])['qtd'].cumsum()
#df_variavel['cumsum_taxa_total'] = df_variavel.groupby(['stock'])['taxa total'].cumsum()
# Group by 'stock' and the month-year combination, and get the last row of each group
#df_acumulado_mes = df_variavel.groupby(['stock', df_variavel['date'].dt.to_period("M")]).apply(lambda group: group.iloc[-1]).reset_index(drop=True).sort_values("date").reset_index(drop=True)
#display(df_acumulado_mes.columns)
#df_acumulado_mes = df_acumulado_mes[['date', 'tipo', 'stock','month', 'year', 'cumsum_total', 'cumsum_qtd']]
#df_acumulado_mes.to_csv("../data/Consolidado/df_ativos_mf.csv",sep=",",index=False)
#df_acumulado_mes
df_variavel = df_variavel.reset_index(drop=True)
df_variavel.to_csv("../data/Consolidado/df_ativos_mf.csv",sep=",",index=False)
df_variavel

,date,tipo,stock,qtd,preço médio,total
0,2022-03-10,ETF,SMAL11.SA,11,110.000000,1210.000000
1,2022-03-10,ETF,BOVA11.SA,11,109.920000,1209.120000
2,2022-03-10,ETF,SMAL11.SA,23,111.370000,2561.510000
3,2022-03-10,ETF,BOVA11.SA,23,112.820000,2594.860000
4,2022-10-18,ETF,VWRA.L,1,502.676683,502.676683
5,2022-10-18,ETF,EIMI.L,2,148.394752,296.789504
6,2022-11-30,ETF,VWRA.L,5,529.357059,2646.785295
7,2023-01-16,ETF,VWRA.L,11,523.129497,5754.424472
8,2023-02-16,ETF,VWRA.L,5,553.891648,2769.458240
9,2023-04-17,ETF,VWRA.L,10,522.478812,5224.788120


In [58]:
# Specify the path where your CSV files are located
path = "../data/downloaded/data_variavel"

# Use glob to get a list of all CSV files in the specified path
all_files = glob.glob(path + "/*.csv")

# Initialize an empty list to store dataframes
dfs = []

# Loop through all CSV files and read them into dataframes
for filename in all_files:
    stock_name = os.path.basename(filename).split("_")[0]
    df = pd.read_csv(filename)
    df["stock"] = stock_name
    #arrumando pq a porra do brew11 da 0 em algum dia do nada
    df["Close"] = df["Close"].replace(0,method="ffill")
    dfs.append(df)


# Concatenate all dataframes into a single dataframe
df_historico = pd.concat(dfs, ignore_index=True)
df_historico = df_historico.rename(
    {"Date": "date", "Close": "close", "Dividends": "dividends"}, axis=1
)
df_historico['date'] = pd.to_datetime(df_historico['date'])

df_historico = df_historico[["date", "close", "dividends", "stock"]]
df_historico["month"] = df_historico["date"].dt.month
df_historico["year"] = df_historico["date"].dt.year
# Print or use the final dataframe as needed
display(df_historico.columns)
#display(df_historico)
# Convert 'date' column to datetime type
df_historico['date'] = pd.to_datetime(df_historico['date'])


# Set the values for rows where 'dividends' is greater than 0
df_historico['date_dividends_payed'] = df_historico.loc[df_historico['dividends'] > 0, 'date']

# Use transform to fill NaN values with the last non-null value within each group
df_historico['date_dividends_payed'] = df_historico.groupby(['stock', 'month', 'year'])['date_dividends_payed'].transform('last')

# Display the resulting DataFrame
display(df_historico)

# Sort DataFrame by 'stock' and 'date'
df_historico.sort_values(by=['stock', 'date'], inplace=True)
#display(df_historico)

df_historico["cumsum_dividendos"] = df_historico.groupby(['stock', 'month', 'year']).agg(
    dividends=pd.NamedAgg(column="dividends", aggfunc="cumsum")
)



# # Create a new column 'end_of_month' to mark the last row of each month
# df_historico['end_of_month'] = df_historico['date'].dt.to_period('M')

# # Select the last row of each month for each stock
# result_df = df_historico.groupby(['stock', 'end_of_month']).last().reset_index()



# Drop the 'end_of_month' column if not needed in the final result
#result_df.drop(columns=['end_of_month'],axis = 1, inplace=True)
#display(df_historico)
# display(result_df)
df_historico.loc[df_historico["dividends"]>0]
df_historico

C:\Users\Pollery\AppData\Local\Temp\ipykernel_5508\2713303913.py:16: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df["Close"] = df["Close"].replace(0,method="ffill")
C:\Users\Pollery\AppData\Local\Temp\ipykernel_5508\2713303913.py:16: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df["Close"] = df["Close"].replace(0,method="ffill")
C:\Users\Pollery\AppData\Local\Temp\ipykernel_5508\2713303913.py:16: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df["Close"] = df["Close"].replace(0,method="ffill")
C:\Users\Pollery\AppData\Local\Temp\ipykernel_5508\2713303913.py:16: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df["Close"] = df["Close"].replace(0,method="ffill")
C:\Users\Pollery\AppData\Local\Temp\ipykernel_5508\2713303913.py:16: Fut

Index(['date', 'close', 'dividends', 'stock', 'month', 'year'], dtype='object')

,date,close,dividends,stock,month,year,date_dividends_payed
0,2000-01-01,NaN,0.0,BOVA11.SA,1,2000,NaT
1,2000-01-02,NaN,0.0,BOVA11.SA,1,2000,NaT
2,2000-01-03,NaN,0.0,BOVA11.SA,1,2000,NaT
3,2000-01-04,NaN,0.0,BOVA11.SA,1,2000,NaT
4,2000-01-05,NaN,0.0,BOVA11.SA,1,2000,NaT
...,...,...,...,...,...,...,...
62060,2024-01-22,90.519997,0.0,WRLD11.SA,1,2024,NaT
62061,2024-01-23,90.040001,0.0,WRLD11.SA,1,2024,NaT
62062,2024-01-24,89.949997,0.0,WRLD11.SA,1,2024,NaT
62063,2024-01-25,90.029999,0.0,WRLD11.SA,1,2024,NaT


,date,close,dividends,stock,month,year,date_dividends_payed,cumsum_dividendos
0,2000-01-01,NaN,0.0,BOVA11.SA,1,2000,NaT,0.0
1,2000-01-02,NaN,0.0,BOVA11.SA,1,2000,NaT,0.0
2,2000-01-03,NaN,0.0,BOVA11.SA,1,2000,NaT,0.0
3,2000-01-04,NaN,0.0,BOVA11.SA,1,2000,NaT,0.0
4,2000-01-05,NaN,0.0,BOVA11.SA,1,2000,NaT,0.0
...,...,...,...,...,...,...,...,...
62060,2024-01-22,90.519997,0.0,WRLD11.SA,1,2024,NaT,0.0
62061,2024-01-23,90.040001,0.0,WRLD11.SA,1,2024,NaT,0.0
62062,2024-01-24,89.949997,0.0,WRLD11.SA,1,2024,NaT,0.0
62063,2024-01-25,90.029999,0.0,WRLD11.SA,1,2024,NaT,0.0


In [59]:
df_historico[df_historico["stock"].str.contains("ITIT11")]

,date,close,dividends,stock,month,year,date_dividends_payed,cumsum_dividendos
26618,2000-01-01,NaN,0.0,ITIT11.SA,1,2000,NaT,0.00
26619,2000-01-02,NaN,0.0,ITIT11.SA,1,2000,NaT,0.00
26620,2000-01-03,NaN,0.0,ITIT11.SA,1,2000,NaT,0.00
26621,2000-01-04,NaN,0.0,ITIT11.SA,1,2000,NaT,0.00
26622,2000-01-05,NaN,0.0,ITIT11.SA,1,2000,NaT,0.00
...,...,...,...,...,...,...,...,...
35405,2024-01-22,78.330002,0.0,ITIT11.SA,1,2024,2024-01-02,0.64
35406,2024-01-23,78.269997,0.0,ITIT11.SA,1,2024,2024-01-02,0.64
35407,2024-01-24,78.440002,0.0,ITIT11.SA,1,2024,2024-01-02,0.64
35408,2024-01-25,78.480003,0.0,ITIT11.SA,1,2024,2024-01-02,0.64


In [60]:
# Generate a date range from January 2020 to today
start_date = datetime(2020, 1, 1)
end_date = datetime.now().date()
date_range = pd.date_range(start=start_date, end=end_date, freq='d')

# Create a DataFrame with 'Month' and 'Year' columns
df_datas = pd.DataFrame({'date': date_range})

#merging all dfs
df_all_stocks = pd.concat([df_acfi, df_ac_int], ignore_index=True)
#df_port = pd.merge(df_datas, df_all_stocks, how="left", on=["date"])
df_port = pd.merge(df_datas, df_historico.drop(["month","year"],axis=1), how="left", on=["date"])
df_port = pd.merge(df_port, df_usd.drop(["month","year"],axis=1), how="left", on=["date"])
df_port["month"] = df_port["date"].dt.month
df_port["year"] = df_port["date"].dt.year
# df_port = df_port.ffill()
df_port.loc[~df_port["stock"].isna()]

,date,close,dividends,stock,date_dividends_payed,cumsum_dividendos,usd_compra,usd_venda,month,year
0,2020-01-01,111.230003,0.0,BOVA11.SA,NaT,0.00,NaN,NaN,1,2020
1,2020-01-01,69.500000,0.0,DIVO11.SA,NaT,0.00,NaN,NaN,1,2020
2,2020-01-01,NaN,0.0,EIMI.L,NaT,0.00,NaN,NaN,1,2020
3,2020-01-01,NaN,0.0,ITIT11.SA,NaT,0.00,NaN,NaN,1,2020
4,2020-01-01,136.440002,0.0,SMAL11.SA,NaT,0.00,NaN,NaN,1,2020
...,...,...,...,...,...,...,...,...,...,...
10925,2024-01-26,30.840000,0.0,EIMI.L,NaT,0.00,NaN,NaN,1,2024
10926,2024-01-26,78.480003,0.0,ITIT11.SA,2024-01-02,0.64,NaN,NaN,1,2024
10927,2024-01-26,107.000000,0.0,SMAL11.SA,NaT,0.00,NaN,NaN,1,2024
10928,2024-01-26,119.379997,0.0,VWRA.L,NaT,0.00,NaN,NaN,1,2024


In [61]:

#arrumando valor de fds do dolar
df_port["usd_compra"] = df_port["usd_compra"].ffill()
df_port["usd_venda"] = df_port["usd_venda"].ffill()

df_port.to_csv("../data/Consolidado/df_historico_total.csv",sep=",",index=False)